<a href="https://colab.research.google.com/github/ejihoon6065/BERT_stock_forecasting/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Sep 17 07:20:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 8.7MB/s 
     |████████████████████████████████| 1.1MB 16.9MB/s 
     |████████████████████████████████| 890kB 28.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4b9acebc7ee7882a3750aecdbee8f973340b9a8a1fd59d4f67b00e16aa66867c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [4]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 16.71 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


그리고, dataframe으로 파일을 읽고 결측치들을 처리해준다. 본 예제에서는 40%만 샘플링하여서 실험하였다.. 너무 많아서 :(

In [5]:
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')

In [6]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=0.4, random_state=999)
test_df = test_df.sample(frac=0.4, random_state=999)

이후 데이터 셋을 만들어주고,

In [7]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

Train에 사용될 DataLoader를 만들어준다. Batch size는 2로 하였는데, 4부터는 Colab의 CUDA 메모리가 터지기 때문이다.. 때문에 굉장히 오래걸리고 NLP가 자원싸움이라는 생각이 많이 들더라.

In [8]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)

이후 과정은 일반적인 LSTM이나 CNN을 사용하는 과정과 동일하다. Huggingface에서 구현된 Bert는 pytorch의 module클래스를 상속받고 있다. 따라서 이미지 분류 classification task처럼 진행해주면 된다.

from_pretrained(‘bert-base-multilingual-cased’)를 사용함으로써 google에서 pretrained한 모델을 사용할 수 있다. 그리고 bert 소개글에서와 같이 tokenizer는 wordpiece를 만들어 토큰화가 이루어진다. 여기서 포인트는 우리가 구현체 중 ‘BertForSequenceClassification’모델을 사용하는 것이다. 이 모델은 디폴트로 이진분류가 되어있다. 따라서 multi-label을 하거나 다른 task를 학습하기 위해서는 구현된 다른 서브 모델들을 활용하면 된다. 예컨대, multi-label을 하고 싶으면 기본 BertModel의 classifier layer를 조정해줘야하는데, 이곳을 참고하면 도움이 될 것이다.

In [9]:
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

100%|██████████| 714314041/714314041 [00:24<00:00, 29337713.78B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

트레이닝 과정은 굉장히 심플하다. optimizer 설정, 일정 주기별로 Loss 찍기 등 흔히 볼 수 있는 코드로 진행이 가능하다. 주의해야하는 것은, 학습 샘플의 인풋이 (batch_size, sequence_length)로 들어간다는 것이다. 따라서 zero-padding을 직접 해줘서 model의 forward에 넣어줘야한다.

In [10]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/1] Iteration 500 -> Train Loss: 0.6968, Accuracy: 0.503
[Epoch 1/1] Iteration 1000 -> Train Loss: 0.6999, Accuracy: 0.495
[Epoch 1/1] Iteration 1500 -> Train Loss: 0.6954, Accuracy: 0.511
[Epoch 1/1] Iteration 2000 -> Train Loss: 0.6948, Accuracy: 0.512
[Epoch 1/1] Iteration 2500 -> Train Loss: 0.6879, Accuracy: 0.538
[Epoch 1/1] Iteration 3000 -> Train Loss: 0.6677, Accuracy: 0.597
[Epoch 1/1] Iteration 3500 -> Train Loss: 0.6456, Accuracy: 0.641
[Epoch 1/1] Iteration 4000 -> Train Loss: 0.6176, Accuracy: 0.680
[Epoch 1/1] Iteration 4500 -> Train Loss: 0.5870, Accuracy: 0.703
[Epoch 1/1] Iteration 5000 -> Train Loss: 0.6023, Accuracy: 0.670
[Epoch 1/1] Iteration 5500 -> Train Loss: 0.5663, Accuracy: 0.711
[Epoch 1/1] Iteration 6000 -> Train Loss: 0.5608, Accuracy: 0.733
[Epoch 1/1] Iteration 6500 -> Train Loss: 0.5336, Accuracy: 0.752
[Epoch 1/1] Iteration 7000 -> Train Loss: 0.5601, Accuracy: 0.705
[Epoch 1/1] Iteration 7500 -> Train Loss: 0.5372, Accuracy: 0.746
[Epoch 1/1]

In [11]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test accuracy:  0.8132406620331016
